In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch
import numpy as np
model_path = "/share/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/home/lzq/miniconda3/envs/react-rl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.12s/it]


In [2]:
from mistral_with_ppo_head import MistralPPOHeadModel
from transformers import MistralConfig
from ppo_head import PPOHead
config = MistralConfig.from_pretrained(model_path)
agent_model = MistralPPOHeadModel(config=config)
embedding_dim = agent_model.model.embed_tokens.embedding_dim
agent_model.set_action_head(PPOHead(input_dim=embedding_dim, output_dim=100))

In [3]:
print(agent_model)

MistralPPOHeadModel(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm

In [4]:
text = "Today weather is sunny"
inputs = tokenizer(text, return_tensors='pt')
inputs['labels'] = torch.randn(1,100)
outputs = agent_model(**inputs)

In [8]:
print(outputs.loss)

tensor(-36.5977, grad_fn=<DivBackward1>)
